In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
(include-book "llm-types")
(include-book "std/lists/top" :dir :system)
(include-book "std/strings/top" :dir :system)


Summary
Form:  ( INCLUDE-BOOK "llm-types" ...)
Rules: NIL
Time:  0.48 seconds (prove: 0.00, print: 0.00, other: 0.48)
 "/workspaces/verified-agent/src/llm-types.lisp"
Note (from clause-processors/equality): disabling DISJOIN, DISJOIN2,
CONJOIN and CONJOIN2.


Summary
Form:  ( INCLUDE-BOOK "std/lists/top" ...)
Rules: NIL
Time:  0.09 seconds (prove: 0.00, print: 0.00, other: 0.09)
 "/home/acl2/books/std/lists/top.lisp"

Summary
Form:  ( INCLUDE-BOOK "std/strings/top" ...)
Rules: NIL
Time:  0.21 seconds (prove: 0.00, print: 0.00, other: 0.21)
 "/home/acl2/books/std/strings/top.lisp"


In [3]:
;; Conservative estimate: ~4 characters per token
;; This is intentionally conservative to ensure we never exceed context
(defconst *chars-per-token* 4)


Summary
Form:  ( DEFCONST *CHARS-PER-TOKEN* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CHARS-PER-TOKEN*


In [4]:
;; Safety margin in tokens for model response generation
(defconst *context-safety-margin* 500)


Summary
Form:  ( DEFCONST *CONTEXT-SAFETY-MARGIN* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CONTEXT-SAFETY-MARGIN*


In [5]:
;; Default truncation limits for tool output (head + tail strategy)
(defconst *output-head-chars* 5000)
(defconst *output-tail-chars* 5000)
(defconst *output-max-chars* 10000)


Summary
Form:  ( DEFCONST *OUTPUT-HEAD-CHARS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *OUTPUT-HEAD-CHARS*

Summary
Form:  ( DEFCONST *OUTPUT-TAIL-CHARS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *OUTPUT-TAIL-CHARS*

Summary
Form:  ( DEFCONST *OUTPUT-MAX-CHARS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *OUTPUT-MAX-CHARS*


In [6]:
;; Estimate tokens from character count 
;; Uses nfix to ensure natp return, and truncate for integer division
(define estimate-tokens ((chars natp))
  :returns (tokens natp)
  ;; chars/4 rounded up = (chars + 3) / 4 using truncate
  (nfix (truncate (+ (nfix chars) (1- *chars-per-token*)) *chars-per-token*)))


Since ESTIMATE-TOKENS is non-recursive, its admission is trivial. 
We observe that the type of ESTIMATE-TOKENS is described by the theorem
(AND (INTEGERP (ESTIMATE-TOKENS CHARS)) (<= 0 (ESTIMATE-TOKENS CHARS))).
We used the :type-prescription rule NFIX.

Computing the guard conjecture for ESTIMATE-TOKENS....

The guard conjecture for ESTIMATE-TOKENS is trivial to prove, given
the :executable-counterparts of EQL and NOT, primitive type reasoning
and the :type-prescription rule NFIX.  ESTIMATE-TOKENS is compliant
with Common Lisp.

Summary
Form:  ( DEFUN ESTIMATE-TOKENS ...)
Rules: ((:EXECUTABLE-COUNTERPART EQL)
        (:EXECUTABLE-COUNTERPART NOT)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:TYPE-PRESCRIPTION NFIX))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM NATP-OF-ESTIMATE-TOKENS
          (B* ((TOKENS (ESTIMATE-TOKENS CHARS)))
            (NATP TOKENS))
          :HINTS ((STD::RETURNSPEC-DEFAULT-DEFAULT-HINT 'ESTIMATE-TOKENS
                   

In [7]:
;; Estimate characters needed for given token count
(define tokens-to-chars ((tokens natp))
  :returns (chars natp)
  (* (nfix tokens) *chars-per-token*))


Since TOKENS-TO-CHARS is non-recursive, its admission is trivial. 
We observe that the type of TOKENS-TO-CHARS is described by the theorem
(OR (AND (INTEGERP (TOKENS-TO-CHARS TOKENS))
         (< 1 (TOKENS-TO-CHARS TOKENS)))
    (EQUAL (TOKENS-TO-CHARS TOKENS) 0)).
We used primitive type reasoning and the :type-prescription rule NFIX.

Computing the guard conjecture for TOKENS-TO-CHARS....

The guard conjecture for TOKENS-TO-CHARS is trivial to prove, given
the :type-prescription rule NFIX.  TOKENS-TO-CHARS is compliant with
Common Lisp.

Summary
Form:  ( DEFUN TOKENS-TO-CHARS ...)
Rules: ((:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:TYPE-PRESCRIPTION NFIX))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM NATP-OF-TOKENS-TO-CHARS
          (B* ((CHARS (TOKENS-TO-CHARS TOKENS)))
            (NATP CHARS))
          :HINTS ((STD::RETURNSPEC-DEFAULT-DEFAULT-HINT 'TOKENS-TO-CHARS
                                                        ID WORLD))
          :RULE-CLAS

In [8]:
;; Get character length of a single message
(define message-char-length ((msg chat-message-p))
  :returns (len natp)
  (length (chat-message->content msg)))


Since MESSAGE-CHAR-LENGTH is non-recursive, its admission is trivial.
We observe that the type of MESSAGE-CHAR-LENGTH is described by the
theorem 
(AND (INTEGERP (MESSAGE-CHAR-LENGTH MSG)) (<= 0 (MESSAGE-CHAR-LENGTH MSG))).
We used the :type-prescription rule LENGTH.

Computing the guard conjecture for MESSAGE-CHAR-LENGTH....

The guard conjecture for MESSAGE-CHAR-LENGTH is trivial to prove, given
the :compound-recognizer rule CONSP-WHEN-CHAT-MESSAGE-P and the :type-
prescription rule CHAT-MESSAGE->CONTENT$INLINE.  MESSAGE-CHAR-LENGTH
is compliant with Common Lisp.

Summary
Form:  ( DEFUN MESSAGE-CHAR-LENGTH ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-CHAT-MESSAGE-P)
        (:TYPE-PRESCRIPTION CHAT-MESSAGE->CONTENT$INLINE)
        (:TYPE-PRESCRIPTION LENGTH))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM NATP-OF-MESSAGE-CHAR-LENGTH
          (B* ((LEN (MESSAGE-CHAR-LENGTH MSG)))
            (NATP LEN))
          :HINTS ((STD::RETURNSPEC-DEFAULT-DEFAU

In [9]:
;; Get total character length of message list (recursive)
(define messages-char-length ((msgs chat-message-list-p))
  :returns (len natp)
  (if (endp msgs)
      0
    (+ (message-char-length (car msgs))
       (messages-char-length (cdr msgs)))))


The admission of MESSAGES-CHAR-LENGTH is trivial, using the relation
O< (which is known to be well-founded on the domain recognized by O-P)
and the measure (ACL2-COUNT MSGS).  We observe that the type of 
MESSAGES-CHAR-LENGTH is described by the theorem 
(AND (INTEGERP (MESSAGES-CHAR-LENGTH MSGS))
     (<= 0 (MESSAGES-CHAR-LENGTH MSGS))).
We used primitive type reasoning and the :type-prescription rule 
MESSAGE-CHAR-LENGTH.

Computing the guard conjecture for MESSAGES-CHAR-LENGTH....

The non-trivial part of the guard conjecture for MESSAGES-CHAR-LENGTH
is

Goal
(IMPLIES (CHAT-MESSAGE-LIST-P MSGS)
         (AND (OR (CONSP MSGS) (EQ MSGS NIL))
              (OR (ENDP MSGS)
                  (CHAT-MESSAGE-P (CAR MSGS)))
              (OR (ENDP MSGS)
                  (CONSP MSGS)
                  (EQUAL MSGS NIL))
              (OR (ENDP MSGS)
                  (CHAT-MESSAGE-LIST-P (CDR MSGS)))
              (OR (ENDP MSGS)
                  (ACL2-NUMBERP (MESSAGE-CHAR-LENGTH (CAR MSGS

In [10]:
;; Estimate total tokens for message list
(define messages-token-estimate ((msgs chat-message-list-p))
  :returns (tokens natp)
  (estimate-tokens (messages-char-length msgs)))


Since MESSAGES-TOKEN-ESTIMATE is non-recursive, its admission is trivial.
We observe that the type of MESSAGES-TOKEN-ESTIMATE is described by
the theorem 
(AND (INTEGERP (MESSAGES-TOKEN-ESTIMATE MSGS))
     (<= 0 (MESSAGES-TOKEN-ESTIMATE MSGS))).
We used the :type-prescription rule ESTIMATE-TOKENS.

Computing the guard conjecture for MESSAGES-TOKEN-ESTIMATE....

The guard conjecture for MESSAGES-TOKEN-ESTIMATE is trivial to prove,
given the :compound-recognizer rules NATP-COMPOUND-RECOGNIZER and 
TRUE-LISTP-WHEN-CHAT-MESSAGE-LIST-P-COMPOUND-RECOGNIZER and the :type-
prescription rule MESSAGES-CHAR-LENGTH.  MESSAGES-TOKEN-ESTIMATE is
compliant with Common Lisp.

Summary
Form:  ( DEFUN MESSAGES-TOKEN-ESTIMATE ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER)
        (:COMPOUND-RECOGNIZER
             TRUE-LISTP-WHEN-CHAT-MESSAGE-LIST-P-COMPOUND-RECOGNIZER)
        (:TYPE-PRESCRIPTION ESTIMATE-TOKENS)
        (:TYPE-PRESCRIPTION MESSAGES-CHAR-LENGTH))
Time:  0.00 seconds (prov

In [11]:
;; Check if adding a message would exceed context limit
(define context-fits-p ((new-msg chat-message-p)
                        (current-msgs chat-message-list-p)
                        (max-tokens natp))
  :returns (fits booleanp)
  (b* ((current-chars (messages-char-length current-msgs))
       (new-chars (message-char-length new-msg))
       (total-chars (+ current-chars new-chars))
       (total-tokens (estimate-tokens total-chars))
       (available (if (> max-tokens *context-safety-margin*)
                      (- max-tokens *context-safety-margin*)
                    0)))
    (< total-tokens available)))


Since CONTEXT-FITS-P is non-recursive, its admission is trivial.  We
observe that the type of CONTEXT-FITS-P is described by the theorem
(OR (EQUAL (CONTEXT-FITS-P NEW-MSG CURRENT-MSGS MAX-TOKENS)
           T)
    (EQUAL (CONTEXT-FITS-P NEW-MSG CURRENT-MSGS MAX-TOKENS)
           NIL)).

Computing the guard conjecture for CONTEXT-FITS-P....

The non-trivial part of the guard conjecture for CONTEXT-FITS-P is

Goal
(IMPLIES
 (AND (NATP MAX-TOKENS)
      (CHAT-MESSAGE-LIST-P CURRENT-MSGS)
      (CHAT-MESSAGE-P NEW-MSG))
 (AND
     (CHAT-MESSAGE-LIST-P CURRENT-MSGS)
     (LET ((CURRENT-CHARS (MESSAGES-CHAR-LENGTH CURRENT-MSGS)))
       (AND (CHAT-MESSAGE-P NEW-MSG)
            (LET ((NEW-CHARS (MESSAGE-CHAR-LENGTH NEW-MSG)))
              (AND (ACL2-NUMBERP CURRENT-CHARS)
                   (ACL2-NUMBERP NEW-CHARS)
                   (LET ((TOTAL-CHARS (+ CURRENT-CHARS NEW-CHARS)))
                     (AND (NATP TOTAL-CHARS)
                          (LET ((TOTAL-TOKENS (ESTIMATE-TOKENS

In [12]:
;; Check if messages fit within token limit
(define messages-fit-p ((msgs chat-message-list-p)
                        (max-tokens natp))
  :returns (fits booleanp)
  (b* ((total-tokens (messages-token-estimate msgs))
       (available (if (> max-tokens *context-safety-margin*)
                      (- max-tokens *context-safety-margin*)
                    0)))
    (<= total-tokens available)))


Since MESSAGES-FIT-P is non-recursive, its admission is trivial.  We
observe that the type of MESSAGES-FIT-P is described by the theorem
(OR (EQUAL (MESSAGES-FIT-P MSGS MAX-TOKENS)
           T)
    (EQUAL (MESSAGES-FIT-P MSGS MAX-TOKENS)
           NIL)).

Computing the guard conjecture for MESSAGES-FIT-P....

The guard conjecture for MESSAGES-FIT-P is trivial to prove, given
the :compound-recognizer rules NATP-COMPOUND-RECOGNIZER and 
TRUE-LISTP-WHEN-CHAT-MESSAGE-LIST-P-COMPOUND-RECOGNIZER, primitive
type reasoning and the :type-prescription rule MESSAGES-TOKEN-ESTIMATE.
MESSAGES-FIT-P is compliant with Common Lisp.

Summary
Form:  ( DEFUN MESSAGES-FIT-P ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER)
        (:COMPOUND-RECOGNIZER
             TRUE-LISTP-WHEN-CHAT-MESSAGE-LIST-P-COMPOUND-RECOGNIZER)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:TYPE-PRESCRIPTION MESSAGES-TOKEN-ESTIMATE))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM BOO

In [13]:
;; Calculate remaining token capacity
(define remaining-tokens ((msgs chat-message-list-p)
                          (max-tokens natp))
  :returns (remaining natp)
  (b* ((used-tokens (messages-token-estimate msgs))
       (available (if (> max-tokens *context-safety-margin*)
                      (- max-tokens *context-safety-margin*)
                    0)))
    (nfix (- available used-tokens))))


Since REMAINING-TOKENS is non-recursive, its admission is trivial.
We observe that the type of REMAINING-TOKENS is described by the theorem
(AND (INTEGERP (REMAINING-TOKENS MSGS MAX-TOKENS))
     (<= 0 (REMAINING-TOKENS MSGS MAX-TOKENS))).
We used the :type-prescription rule NFIX.

Computing the guard conjecture for REMAINING-TOKENS....

The guard conjecture for REMAINING-TOKENS is trivial to prove, given
the :compound-recognizer rules NATP-COMPOUND-RECOGNIZER and 
TRUE-LISTP-WHEN-CHAT-MESSAGE-LIST-P-COMPOUND-RECOGNIZER, primitive
type reasoning and the :type-prescription rule MESSAGES-TOKEN-ESTIMATE.
REMAINING-TOKENS is compliant with Common Lisp.

Summary
Form:  ( DEFUN REMAINING-TOKENS ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER)
        (:COMPOUND-RECOGNIZER
             TRUE-LISTP-WHEN-CHAT-MESSAGE-LIST-P-COMPOUND-RECOGNIZER)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:TYPE-PRESCRIPTION MESSAGES-TOKEN-ESTIMATE)
        (:TYPE-PRESCRIPTION NFIX))
Time:  0.01 se

In [14]:
;; Check if first message is a system message
(define first-is-system-p ((msgs chat-message-list-p))
  :returns (is-system booleanp)
  (if (endp msgs)
      nil
    (let ((role (chat-message->role (car msgs))))
      (chat-role-case role
        :system t
        :otherwise nil))))


Since FIRST-IS-SYSTEM-P is non-recursive, its admission is trivial.
We observe that the type of FIRST-IS-SYSTEM-P is described by the theorem
(OR (EQUAL (FIRST-IS-SYSTEM-P MSGS) T) (EQUAL (FIRST-IS-SYSTEM-P MSGS) NIL)).
We used primitive type reasoning.

Computing the guard conjecture for FIRST-IS-SYSTEM-P....

The non-trivial part of the guard conjecture for FIRST-IS-SYSTEM-P,
given the :executable-counterpart of EQLABLEP, is

Goal
(IMPLIES (CHAT-MESSAGE-LIST-P MSGS)
         (AND (OR (CONSP MSGS) (EQ MSGS NIL))
              (OR (ENDP MSGS)
                  (CONSP MSGS)
                  (EQUAL MSGS NIL))
              (OR (ENDP MSGS)
                  (CHAT-MESSAGE-P (CAR MSGS)))
              (OR (ENDP MSGS)
                  (LET ((ROLE (CHAT-MESSAGE->ROLE (CAR MSGS))))
                    (AND (CHAT-ROLE-P ROLE)
                         (LET ((ROLE.KIND (CHAT-ROLE-KIND ROLE)))
                           T)))))).
Goal'

Q.E.D.

That completes the proof of the guard theorem for F

In [15]:
;; Extract system message (first message if system, nil otherwise)
(define extract-system-message ((msgs chat-message-list-p))
  :returns (sys-msg (or (null sys-msg) (chat-message-p sys-msg))
                    :hints (("Goal" :in-theory (enable first-is-system-p))))
  (if (and (consp msgs) (first-is-system-p msgs))
      (chat-message-fix (car msgs))
    nil))


Since EXTRACT-SYSTEM-MESSAGE is non-recursive, its admission is trivial.
We observe that the type of EXTRACT-SYSTEM-MESSAGE is described by
the theorem (TRUE-LISTP (EXTRACT-SYSTEM-MESSAGE MSGS)).  We used the
:type-prescription rule CHAT-MESSAGE-FIX$INLINE.

Computing the guard conjecture for EXTRACT-SYSTEM-MESSAGE....

The non-trivial part of the guard conjecture for EXTRACT-SYSTEM-MESSAGE
is

Goal
(IMPLIES (CHAT-MESSAGE-LIST-P MSGS)
         (AND (OR (NOT (CONSP MSGS))
                  (CHAT-MESSAGE-LIST-P MSGS))
              (OR (NOT (AND (CONSP MSGS)
                            (FIRST-IS-SYSTEM-P MSGS)))
                  (CONSP MSGS)
                  (EQUAL MSGS NIL))
              (OR (NOT (AND (CONSP MSGS)
                            (FIRST-IS-SYSTEM-P MSGS)))
                  (CHAT-MESSAGE-P (CAR MSGS))))).
Goal'

Q.E.D.

That completes the proof of the guard theorem for EXTRACT-SYSTEM-MESSAGE.
EXTRACT-SYSTEM-MESSAGE is compliant with Common Lisp.

Summary
Form:  ( DEFUN E

In [16]:
;; Get non-system messages (rest if first is system, all otherwise)
(define non-system-messages ((msgs chat-message-list-p))
  :returns (rest chat-message-list-p
                 :hints (("Goal" :in-theory (enable first-is-system-p))))
  (if (and (consp msgs) (first-is-system-p msgs))
      (chat-message-list-fix (cdr msgs))
    (chat-message-list-fix msgs)))


Since NON-SYSTEM-MESSAGES is non-recursive, its admission is trivial.
We observe that the type of NON-SYSTEM-MESSAGES is described by the
theorem (TRUE-LISTP (NON-SYSTEM-MESSAGES MSGS)).  We used the :type-
prescription rule CHAT-MESSAGE-LIST-FIX$INLINE.

Computing the guard conjecture for NON-SYSTEM-MESSAGES....

The non-trivial part of the guard conjecture for NON-SYSTEM-MESSAGES
is

Goal
(IMPLIES (CHAT-MESSAGE-LIST-P MSGS)
         (AND (OR (NOT (CONSP MSGS))
                  (CHAT-MESSAGE-LIST-P MSGS))
              (OR (NOT (AND (CONSP MSGS)
                            (FIRST-IS-SYSTEM-P MSGS)))
                  (CONSP MSGS)
                  (EQUAL MSGS NIL))
              (OR (NOT (AND (CONSP MSGS)
                            (FIRST-IS-SYSTEM-P MSGS)))
                  (CHAT-MESSAGE-LIST-P (CDR MSGS)))
              (OR (AND (CONSP MSGS)
                       (FIRST-IS-SYSTEM-P MSGS))
                  (CHAT-MESSAGE-LIST-P MSGS)))).
Goal'

Q.E.D.

That completes the proof o

In [17]:
;; Drop oldest messages until list fits in token budget
;; Note: This preserves order - oldest are dropped first
(define drop-oldest-until-fit ((msgs chat-message-list-p)
                                (max-tokens natp))
  :returns (truncated chat-message-list-p)
  :measure (len msgs)
  (b* ((msgs (chat-message-list-fix msgs)))
    (if (endp msgs)
        nil
      (if (messages-fit-p msgs max-tokens)
          msgs
        ;; Drop oldest (first) and try again
        (drop-oldest-until-fit (cdr msgs) max-tokens)))))


For the admission of DROP-OLDEST-UNTIL-FIT we will use the relation
O< (which is known to be well-founded on the domain recognized by O-P)
and the measure (LEN MSGS).  The non-trivial part of the measure conjecture
is

Goal
(AND (O-P (LEN MSGS))
     (IMPLIES (AND (NOT (ENDP (CHAT-MESSAGE-LIST-FIX MSGS)))
                   (NOT (MESSAGES-FIT-P (CHAT-MESSAGE-LIST-FIX MSGS)
                                        MAX-TOKENS)))
              (O< (LEN (CDR (CHAT-MESSAGE-LIST-FIX MSGS)))
                  (LEN MSGS)))).
Goal'
Goal''

([ A key checkpoint:

Goal''
(IMPLIES (AND (CONSP MSGS)
              (NOT (MESSAGES-FIT-P (CHAT-MESSAGE-LIST-FIX MSGS)
                                   MAX-TOKENS)))
         (< (LEN (CDR (CHAT-MESSAGE-LIST-FIX MSGS)))
            (LEN MSGS)))

*1 (Goal'') is pushed for proof by induction.

])

Perhaps we can prove *1 by induction.  One induction scheme is suggested
by this conjecture.  

We will induct according to a scheme suggested by (LEN MSGS).

This 

In [18]:
;; Truncate conversation keeping system prompt + most recent messages
;; Strategy: If first message is system, preserve it and truncate the rest
(define truncate-to-fit ((msgs chat-message-list-p)
                         (max-tokens natp))
  :returns (truncated chat-message-list-p)
  :guard-hints (("Goal" :in-theory (enable extract-system-message
                                            first-is-system-p)))
  (b* ((sys-msg (extract-system-message msgs))
       (rest-msgs (non-system-messages msgs)))
    (if sys-msg
        ;; Have system prompt - calculate remaining budget for conversation
        (b* ((sys-tokens (messages-token-estimate (list sys-msg)))
             ;; max-tokens - sys-tokens - safety-margin
             (rest-budget (nfix (- (nfix (- max-tokens sys-tokens)) 
                                   *context-safety-margin*)))
             (trimmed-rest (drop-oldest-until-fit rest-msgs rest-budget)))
          (cons (chat-message-fix sys-msg) trimmed-rest))
      ;; No system prompt - just truncate everything
      (drop-oldest-until-fit msgs max-tokens))))


Since TRUNCATE-TO-FIT is non-recursive, its admission is trivial. 
We observe that the type of TRUNCATE-TO-FIT is described by the theorem
(TRUE-LISTP (TRUNCATE-TO-FIT MSGS MAX-TOKENS)).  We used primitive
type reasoning and the :type-prescription rule DROP-OLDEST-UNTIL-FIT.

Computing the guard conjecture for TRUNCATE-TO-FIT....

The non-trivial part of the guard conjecture for TRUNCATE-TO-FIT is

Goal
(IMPLIES
 (AND (NATP MAX-TOKENS)
      (CHAT-MESSAGE-LIST-P MSGS))
 (AND
  (CHAT-MESSAGE-LIST-P MSGS)
  (LET ((SYS-MSG (EXTRACT-SYSTEM-MESSAGE MSGS)))
   (AND
    (CHAT-MESSAGE-LIST-P MSGS)
    (LET ((REST-MSGS (NON-SYSTEM-MESSAGES MSGS)))
     (AND
      (OR (NOT SYS-MSG)
          (CHAT-MESSAGE-LIST-P (LIST SYS-MSG)))
      (OR
       (NOT SYS-MSG)
       (LET ((SYS-TOKENS (MESSAGES-TOKEN-ESTIMATE (LIST SYS-MSG))))
        (AND (ACL2-NUMBERP SYS-TOKENS)
             (ACL2-NUMBERP MAX-TOKENS)
             (ACL2-NUMBERP (- SYS-TOKENS))
             (ACL2-NUMBERP (NFIX (+ MAX-TOKENS (- 

In [19]:
;; Helper: nthcdr preserves chat-message-list-p
(defthm chat-message-list-p-of-nthcdr
  (implies (chat-message-list-p msgs)
           (chat-message-list-p (nthcdr n msgs)))
  :hints (("Goal" :in-theory (enable nthcdr))))


*1 (the initial Goal, a key checkpoint) is pushed for proof by induction.

Perhaps we can prove *1 by induction.  One induction scheme is suggested
by this conjecture.  

We will induct according to a scheme suggested by (NTHCDR N MSGS).

This suggestion was produced using the :induction rule NTHCDR.  If
we let (:P MSGS N) denote *1 above then the induction scheme we'll
use is
(AND (IMPLIES (AND (NOT (ZP N))
                   (:P (CDR MSGS) (+ N -1)))
              (:P MSGS N))
     (IMPLIES (ZP N) (:P MSGS N))).
This induction is justified by the same argument used to admit NTHCDR.
Note, however, that the unmeasured variable MSGS is being instantiated.
When applied to the goal at hand the above induction scheme produces
three nontautological subgoals.
Subgoal *1/3
Subgoal *1/3'
Subgoal *1/3''
Subgoal *1/3'''
Subgoal *1/2
Subgoal *1/1

*1 is COMPLETED!
Thus key checkpoint Goal is COMPLETED!

Q.E.D.

The storage of CHAT-MESSAGE-LIST-P-OF-NTHCDR depends upon the :compound-
recognizer r

In [20]:
;; Keep only the most recent N messages (plus system prompt if present)
(define keep-recent-n ((msgs chat-message-list-p)
                       (n natp))
  :returns (recent chat-message-list-p)
  :guard-hints (("Goal" :in-theory (enable extract-system-message
                                            first-is-system-p)))
  (b* ((sys-msg (extract-system-message msgs))
       (rest-msgs (non-system-messages msgs))
       (rest-len (len rest-msgs))
       (skip-count (if (> rest-len n) (- rest-len n) 0))
       (recent-rest (chat-message-list-fix (nthcdr skip-count rest-msgs))))
    (if sys-msg
        (cons (chat-message-fix sys-msg) recent-rest)
      recent-rest)))


Since KEEP-RECENT-N is non-recursive, its admission is trivial.  We
observe that the type of KEEP-RECENT-N is described by the theorem
(TRUE-LISTP (KEEP-RECENT-N MSGS N)).  We used primitive type reasoning
and the :type-prescription rule CHAT-MESSAGE-LIST-FIX$INLINE.

Computing the guard conjecture for KEEP-RECENT-N....

The non-trivial part of the guard conjecture for KEEP-RECENT-N is

Goal
(IMPLIES
 (AND (NATP N)
      (CHAT-MESSAGE-LIST-P MSGS))
 (AND
     (CHAT-MESSAGE-LIST-P MSGS)
     (LET ((SYS-MSG (EXTRACT-SYSTEM-MESSAGE MSGS)))
       (AND (CHAT-MESSAGE-LIST-P MSGS)
            (LET* ((REST-MSGS (NON-SYSTEM-MESSAGES MSGS))
                   (REST-LEN (LEN REST-MSGS)))
              (AND (RATIONALP N)
                   (RATIONALP REST-LEN)
                   (OR (<= REST-LEN N) (ACL2-NUMBERP N))
                   (OR (<= REST-LEN N)
                       (ACL2-NUMBERP REST-LEN))
                   (OR (<= REST-LEN N)
                       (ACL2-NUMBERP (- N)))
           

In [21]:
;; Truncate a long string using head + tail + elision marker
;; Based on mini-swe-agent's output truncation pattern
;; Note: Guard verification deferred - runtime bounds checking via nfix
(define truncate-output ((output stringp)
                         (max-chars natp))
  :returns (truncated stringp)
  (b* ((output (str-fix output))
       (len (length output)))
    (if (<= len max-chars)
        output
      ;; Use head + tail strategy
      (b* ((head-len (nfix (min *output-head-chars* (floor max-chars 2))))
           ;; (max-chars - head-len - 50) for separator overhead
           (tail-budget (nfix (- (nfix (- max-chars head-len)) 50)))
           (tail-len (nfix (min *output-tail-chars* tail-budget)))
           (head-part (subseq output 0 (min head-len len)))
           (tail-start (nfix (max head-len (- len tail-len))))
           (tail-part (subseq output (min tail-start len) len))
           (elided (nfix (- tail-start head-len))))
        (str::cat head-part
                  (str::cat "

[... " 
                            (str::cat (str::nat-to-dec-string elided)
                                      (str::cat " characters elided ...]

"
                                              tail-part))))))))


Since TRUNCATE-OUTPUT is non-recursive, its admission is trivial. 
We observe that the type of TRUNCATE-OUTPUT is described by the theorem
(STRINGP (TRUNCATE-OUTPUT OUTPUT MAX-CHARS)).  We used the :type-prescription
rules STR::FAST-STRING-APPEND and STRINGP-OF-STR-FIX.

Computing the guard conjecture for TRUNCATE-OUTPUT....

The non-trivial part of the guard conjecture for TRUNCATE-OUTPUT, given
the :executable-counterparts of EQL and NOT, is

Goal
(IMPLIES
 (AND (NATP MAX-CHARS) (STRINGP OUTPUT))
 (LET ((OUTPUT (STR-FIX OUTPUT)))
  (AND
   (OR (TRUE-LISTP OUTPUT)
       (STRINGP OUTPUT))
   (LET ((LEN (LENGTH OUTPUT)))
    (AND
     (RATIONALP MAX-CHARS)
     (RATIONALP LEN)
     (OR (<= LEN MAX-CHARS)
         (RATIONALP MAX-CHARS))
     (OR (<= LEN MAX-CHARS)
         (RATIONALP (FLOOR MAX-CHARS 2)))
     (OR
      (<= LEN MAX-CHARS)
      (LET ((HEAD-LEN (NFIX (MIN 5000 (FLOOR MAX-CHARS 2)))))
       (AND
        (ACL2-NUMBERP HEAD-LEN)
        (ACL2-NUMBERP MAX-CHARS)
        (A

In [22]:
;; Helper: append preserves chat-message-list-p (by induction on first arg)
(defthm chat-message-list-p-of-append
  (implies (and (chat-message-list-p x)
                (chat-message-list-p y))
           (chat-message-list-p (append x y)))
  :hints (("Goal" :in-theory (enable append chat-message-list-p))))


*1 (the initial Goal, a key checkpoint) is pushed for proof by induction.

Perhaps we can prove *1 by induction.  Three induction schemes are
suggested by this conjecture.  These merge into two derived induction
schemes.  However, one of these is flawed and so we are left with one
viable candidate.  

We will induct according to a scheme suggested by (APPEND X Y), while
accommodating (CHAT-MESSAGE-LIST-P X).

These suggestions were produced using the :induction rules BINARY-APPEND
and CHAT-MESSAGE-LIST-P.  If we let (:P X Y) denote *1 above then the
induction scheme we'll use is
(AND (IMPLIES (AND (NOT (ENDP X)) (:P (CDR X) Y))
              (:P X Y))
     (IMPLIES (ENDP X) (:P X Y))).
This induction is justified by the same argument used to admit BINARY-APPEND.
When applied to the goal at hand the above induction scheme produces
three nontautological subgoals.
Subgoal *1/3
Subgoal *1/3'

Splitter note (see :DOC splitter) for Subgoal *1/3' (2 subgoals).
  if-intro: ((:DEFINITION CHAT-

In [23]:
;; Add a message to conversation with automatic truncation if needed
(define add-message ((msg chat-message-p)
                     (msgs chat-message-list-p)
                     (max-tokens natp))
  :returns (new-msgs chat-message-list-p)
  (b* ((msgs (chat-message-list-fix msgs))
       (msg (chat-message-fix msg))
       (new-msgs (append msgs (list msg))))
    (if (messages-fit-p new-msgs max-tokens)
        new-msgs
      ;; Truncate to fit
      (truncate-to-fit new-msgs max-tokens))))


Since ADD-MESSAGE is non-recursive, its admission is trivial.  We observe
that the type of ADD-MESSAGE is described by the theorem 
(TRUE-LISTP (ADD-MESSAGE MSG MSGS MAX-TOKENS)).  We used primitive
type reasoning and the :type-prescription rules BINARY-APPEND, 
TRUE-LISTP-APPEND and TRUNCATE-TO-FIT.

Computing the guard conjecture for ADD-MESSAGE....

The non-trivial part of the guard conjecture for ADD-MESSAGE is

Goal
(IMPLIES (AND (NATP MAX-TOKENS)
              (CHAT-MESSAGE-LIST-P MSGS)
              (CHAT-MESSAGE-P MSG))
         (AND (CHAT-MESSAGE-LIST-P MSGS)
              (LET ((MSGS (CHAT-MESSAGE-LIST-FIX MSGS)))
                (AND (CHAT-MESSAGE-P MSG)
                     (LET ((MSG (CHAT-MESSAGE-FIX MSG)))
                       (AND (TRUE-LISTP MSGS)
                            (LET ((NEW-MSGS (APPEND MSGS (LIST MSG))))
                              (AND (CHAT-MESSAGE-LIST-P NEW-MSGS)
                                   (NATP MAX-TOKENS)
                                

In [24]:
;; Add user message
(define add-user-message ((content stringp)
                          (msgs chat-message-list-p)
                          (max-tokens natp))
  :returns (new-msgs chat-message-list-p)
  (add-message (make-user-message content) msgs max-tokens))


Since ADD-USER-MESSAGE is non-recursive, its admission is trivial.
We observe that the type of ADD-USER-MESSAGE is described by the theorem
(TRUE-LISTP (ADD-USER-MESSAGE CONTENT MSGS MAX-TOKENS)).  We used the
:type-prescription rule ADD-MESSAGE.

Computing the guard conjecture for ADD-USER-MESSAGE....

The non-trivial part of the guard conjecture for ADD-USER-MESSAGE is

Goal
(IMPLIES (AND (NATP MAX-TOKENS)
              (CHAT-MESSAGE-LIST-P MSGS)
              (STRINGP CONTENT))
         (AND (STRINGP CONTENT)
              (CHAT-MESSAGE-P (MAKE-USER-MESSAGE CONTENT))
              (NATP MAX-TOKENS)
              (CHAT-MESSAGE-LIST-P MSGS))).

Q.E.D.

That completes the proof of the guard theorem for ADD-USER-MESSAGE.
ADD-USER-MESSAGE is compliant with Common Lisp.

Summary
Form:  ( DEFUN ADD-USER-MESSAGE ...)
Rules: ((:DEFINITION NATP)
        (:DEFINITION NOT)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM)
        (:REWRITE CHAT-MESSAGE-P-OF-MAKE-USER-MESSAGE)
        (:TYPE-PRESCRI

In [25]:
;; Add assistant message  
(define add-assistant-message ((content stringp)
                               (msgs chat-message-list-p)
                               (max-tokens natp))
  :returns (new-msgs chat-message-list-p)
  (add-message (make-assistant-message content) msgs max-tokens))


Since ADD-ASSISTANT-MESSAGE is non-recursive, its admission is trivial.
We observe that the type of ADD-ASSISTANT-MESSAGE is described by the
theorem (TRUE-LISTP (ADD-ASSISTANT-MESSAGE CONTENT MSGS MAX-TOKENS)).
We used the :type-prescription rule ADD-MESSAGE.

Computing the guard conjecture for ADD-ASSISTANT-MESSAGE....

The non-trivial part of the guard conjecture for ADD-ASSISTANT-MESSAGE
is

Goal
(IMPLIES (AND (NATP MAX-TOKENS)
              (CHAT-MESSAGE-LIST-P MSGS)
              (STRINGP CONTENT))
         (AND (STRINGP CONTENT)
              (CHAT-MESSAGE-P (MAKE-ASSISTANT-MESSAGE CONTENT))
              (NATP MAX-TOKENS)
              (CHAT-MESSAGE-LIST-P MSGS))).

Q.E.D.

That completes the proof of the guard theorem for ADD-ASSISTANT-MESSAGE.
ADD-ASSISTANT-MESSAGE is compliant with Common Lisp.

Summary
Form:  ( DEFUN ADD-ASSISTANT-MESSAGE ...)
Rules: ((:DEFINITION NATP)
        (:DEFINITION NOT)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM)
        (:REWRITE CHAT-MESSAGE-P-

In [26]:
;; Add tool result message (with optional output truncation)
(define add-tool-message ((content stringp)
                          (msgs chat-message-list-p)
                          (max-tokens natp)
                          (truncate-output-p booleanp))
  :returns (new-msgs chat-message-list-p)
  (b* ((content-final (if truncate-output-p
                          (truncate-output content *output-max-chars*)
                        content)))
    (add-message (make-tool-message content-final) msgs max-tokens)))


Since ADD-TOOL-MESSAGE is non-recursive, its admission is trivial.
We observe that the type of ADD-TOOL-MESSAGE is described by the theorem
(TRUE-LISTP (ADD-TOOL-MESSAGE CONTENT MSGS MAX-TOKENS TRUNCATE-OUTPUT-P)).
We used the :type-prescription rule ADD-MESSAGE.

Computing the guard conjecture for ADD-TOOL-MESSAGE....

The non-trivial part of the guard conjecture for ADD-TOOL-MESSAGE,
given the :executable-counterpart of NATP, is

Goal
(IMPLIES
 (AND (BOOLEANP TRUNCATE-OUTPUT-P)
      (NATP MAX-TOKENS)
      (CHAT-MESSAGE-LIST-P MSGS)
      (STRINGP CONTENT))
 (AND
   (OR (NOT TRUNCATE-OUTPUT-P)
       (STRINGP CONTENT))
   (LET ((CONTENT-FINAL (IF TRUNCATE-OUTPUT-P (TRUNCATE-OUTPUT CONTENT 10000)
                          CONTENT)))
     (AND (STRINGP CONTENT-FINAL)
          (CHAT-MESSAGE-P (MAKE-TOOL-MESSAGE CONTENT-FINAL))
          (NATP MAX-TOKENS)
          (CHAT-MESSAGE-LIST-P MSGS))))).

Q.E.D.

That completes the proof of the guard theorem for ADD-TOOL-MESSAGE.
ADD-TOOL-MES

In [27]:
;; Initialize conversation with system prompt
(define init-conversation ((system-prompt stringp)
                           (max-tokens natp))
  :returns (msgs chat-message-list-p)
  (b* ((sys-msg (make-system-message system-prompt)))
    (if (messages-fit-p (list sys-msg) max-tokens)
        (list sys-msg)
      ;; System prompt too long - truncate it
      (list (make-system-message 
              (truncate-output system-prompt 
                               (tokens-to-chars max-tokens)))))))


Since INIT-CONVERSATION is non-recursive, its admission is trivial.
We observe that the type of INIT-CONVERSATION is described by the theorem
(AND (CONSP (INIT-CONVERSATION SYSTEM-PROMPT MAX-TOKENS))
     (TRUE-LISTP (INIT-CONVERSATION SYSTEM-PROMPT MAX-TOKENS))).
We used primitive type reasoning.

Computing the guard conjecture for INIT-CONVERSATION....

The non-trivial part of the guard conjecture for INIT-CONVERSATION
is

Goal
(IMPLIES
 (AND (NATP MAX-TOKENS)
      (STRINGP SYSTEM-PROMPT))
 (AND
    (STRINGP SYSTEM-PROMPT)
    (LET ((SYS-MSG (MAKE-SYSTEM-MESSAGE SYSTEM-PROMPT)))
      (AND (CHAT-MESSAGE-LIST-P (LIST SYS-MSG))
           (NATP MAX-TOKENS)
           (OR (MESSAGES-FIT-P (LIST SYS-MSG)
                               MAX-TOKENS)
               (NATP MAX-TOKENS))
           (OR (MESSAGES-FIT-P (LIST SYS-MSG)
                               MAX-TOKENS)
               (STRINGP SYSTEM-PROMPT))
           (OR (MESSAGES-FIT-P (LIST SYS-MSG)
                               MAX-

In [28]:
;; Helper lemma: append preserves chat-message-list-p
(defthm chat-message-list-p-of-append
  (implies (and (chat-message-list-p x)
                (chat-message-list-p y))
           (chat-message-list-p (append x y)))
  :hints (("Goal" :induct (true-listp x))))


The event ( DEFTHM CHAT-MESSAGE-LIST-P-OF-APPEND ...) is redundant.
See :DOC redundant-events.

Summary
Form:  ( DEFTHM CHAT-MESSAGE-LIST-P-OF-APPEND ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [29]:
;; Theorem: drop-oldest-until-fit returns a sublist
(defthm drop-oldest-until-fit-is-sublist
  (implies (chat-message-list-p msgs)
           (subsetp-equal (drop-oldest-until-fit msgs max-tokens) msgs))
  :hints (("Goal" :in-theory (enable drop-oldest-until-fit))))


*1 (the initial Goal, a key checkpoint) is pushed for proof by induction.

Perhaps we can prove *1 by induction.  One induction scheme is suggested
by this conjecture.  

We will induct according to a scheme suggested by 
(DROP-OLDEST-UNTIL-FIT MSGS MAX-TOKENS).

This suggestion was produced using the :induction rule DROP-OLDEST-UNTIL-FIT.
If we let (:P MAX-TOKENS MSGS) denote *1 above then the induction scheme
we'll use is
(AND (IMPLIES (AND (NOT (ENDP (CHAT-MESSAGE-LIST-FIX MSGS)))
                   (NOT (MESSAGES-FIT-P (CHAT-MESSAGE-LIST-FIX MSGS)
                                        MAX-TOKENS))
                   (:P MAX-TOKENS
                       (CDR (CHAT-MESSAGE-LIST-FIX MSGS))))
              (:P MAX-TOKENS MSGS))
     (IMPLIES (AND (NOT (ENDP (CHAT-MESSAGE-LIST-FIX MSGS)))
                   (MESSAGES-FIT-P (CHAT-MESSAGE-LIST-FIX MSGS)
                                   MAX-TOKENS))
              (:P MAX-TOKENS MSGS))
     (IMPLIES (ENDP (CHAT-MESSAGE-LIST-FIX MSGS))

In [30]:
;; Helper: drop-oldest-until-fit length bound (using chat-message-list-p hypothesis)
(defthm drop-oldest-until-fit-length-bound
  (implies (chat-message-list-p msgs)
           (<= (len (drop-oldest-until-fit msgs max-tokens))
               (len msgs)))
  :hints (("Goal" :in-theory (enable drop-oldest-until-fit)))
  :rule-classes :linear)


ACL2 Warning [Double-rewrite] in ( DEFTHM DROP-OLDEST-UNTIL-FIT-LENGTH-BOUND
...):  In a :LINEAR rule generated from DROP-OLDEST-UNTIL-FIT-LENGTH-BOUND
for trigger term (LEN (DROP-OLDEST-UNTIL-FIT MSGS MAX-TOKENS)), equivalence
relations STR::ICHARLISTEQV, STR::CHARLISTEQV$INLINE and LIST-EQUIV
are maintained at one problematic occurrence of variable MSGS in the
conclusion, but not at any binding occurrence of MSGS.  Consider replacing
that occurrence of MSGS in the conclusion with (DOUBLE-REWRITE MSGS).
See :doc double-rewrite for more information on this issue.


*1 (the initial Goal, a key checkpoint) is pushed for proof by induction.

Perhaps we can prove *1 by induction.  Two induction schemes are suggested
by this conjecture.  By considering those suggested by the largest
number of non-primitive recursive functions, we narrow the field to
one.  

We will induct according to a scheme suggested by 
(DROP-OLDEST-UNTIL-FIT MSGS MAX-TOKENS).

This suggestion was produced using the :i

In [31]:
;; Helper: drop-oldest-until-fit of nil is nil
(defthm drop-oldest-until-fit-of-nil
  (equal (drop-oldest-until-fit nil max-tokens) nil)
  :hints (("Goal" :in-theory (enable drop-oldest-until-fit))))

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM DROP-OLDEST-UNTIL-FIT-OF-NIL ...)
Rules: ((:DEFINITION DROP-OLDEST-UNTIL-FIT)
        (:DEFINITION NOT)
        (:EXECUTABLE-COUNTERPART CHAT-MESSAGE-LIST-FIX$INLINE)
        (:EXECUTABLE-COUNTERPART CONSP))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.01)
Prover steps counted:  42
 DROP-OLDEST-UNTIL-FIT-OF-NIL


In [32]:
;; Theorem: truncated list has no more elements than original
(defthm truncate-to-fit-length-bound
  (implies (chat-message-list-p msgs)
           (<= (len (truncate-to-fit msgs max-tokens))
               (len msgs)))
  :hints (("Goal" :in-theory (enable truncate-to-fit 
                                      non-system-messages
                                      extract-system-message
                                      first-is-system-p))))

Goal'
Subgoal 3
Subgoal 3'
Subgoal 2

Splitter note (see :DOC splitter) for Subgoal 2 (4 subgoals).
  if-intro: ((:DEFINITION NFIX))

Subgoal 2.4
Subgoal 2.3
Subgoal 2.3'
Subgoal 2.3''
Subgoal 2.3.2
Subgoal 2.3.1
Subgoal 2.3.1'
Subgoal 2.2
Subgoal 2.2'

Splitter note (see :DOC splitter) for Subgoal 2.2' (2 subgoals).
  if-intro: ((:DEFINITION NFIX))

Subgoal 2.2.2
Subgoal 2.2.2.2
Subgoal 2.2.2.1
Subgoal 2.2.2.1'
Subgoal 2.2.1
Subgoal 2.2.1.2
Subgoal 2.2.1.1
Subgoal 2.2.1.1'
Subgoal 2.1
Subgoal 2.1'
Subgoal 2.1''
Subgoal 2.1.2
Subgoal 2.1.1
Subgoal 2.1.1'
Subgoal 1
Subgoal 1'

Q.E.D.

Summary
Form:  ( DEFTHM TRUNCATE-TO-FIT-LENGTH-BOUND ...)
Rules: ((:COMPOUND-RECOGNIZER
             TRUE-LISTP-WHEN-CHAT-MESSAGE-LIST-P-COMPOUND-RECOGNIZER)
        (:DEFINITION DOUBLE-REWRITE)
        (:DEFINITION EXTRACT-SYSTEM-MESSAGE)
        (:DEFINITION FIRST-IS-SYSTEM-P)
        (:DEFINITION NFIX)
        (:DEFINITION NON-SYSTEM-MESSAGES)
        (:DEFINITION NOT)
        (:DEFINITION TRUNCATE-TO-F

In [33]:
;; Theorem: messages-char-length is non-negative
(defthm messages-char-length-natp
  (natp (messages-char-length msgs))
  :hints (("Goal" :in-theory (enable messages-char-length)))
  :rule-classes :type-prescription)


ACL2 Observation in ( DEFTHM MESSAGES-CHAR-LENGTH-NATP ...):  Our heuristics
choose (MESSAGES-CHAR-LENGTH MSGS) as the :TYPED-TERM.

Q.E.D.

The storage of MESSAGES-CHAR-LENGTH-NATP depends upon the :compound-
recognizer rule NATP-COMPOUND-RECOGNIZER.

Summary
Form:  ( DEFTHM MESSAGES-CHAR-LENGTH-NATP ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER)
        (:TYPE-PRESCRIPTION MESSAGES-CHAR-LENGTH))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.01)
 MESSAGES-CHAR-LENGTH-NATP


In [34]:
;; Theorem: estimate-tokens is non-negative
(defthm estimate-tokens-natp
  (implies (natp chars)
           (natp (estimate-tokens chars)))
  :hints (("Goal" :in-theory (enable estimate-tokens)))
  :rule-classes :type-prescription)


ACL2 Observation in ( DEFTHM ESTIMATE-TOKENS-NATP ...):  Our heuristics
choose (ESTIMATE-TOKENS CHARS) as the :TYPED-TERM.

Q.E.D.

The storage of ESTIMATE-TOKENS-NATP depends upon the :compound-recognizer
rule NATP-COMPOUND-RECOGNIZER.

Summary
Form:  ( DEFTHM ESTIMATE-TOKENS-NATP ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER)
        (:TYPE-PRESCRIPTION ESTIMATE-TOKENS))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.01)
 ESTIMATE-TOKENS-NATP


In [35]:
;; Theorem: add-message returns a chat-message-list
(defthm add-message-returns-list
  (implies (and (chat-message-p msg)
                (chat-message-list-p msgs)
                (natp max-tokens))
           (chat-message-list-p (add-message msg msgs max-tokens)))
  :hints (("Goal" :in-theory (enable add-message))))


ACL2 Warning [Subsume] in ( DEFTHM ADD-MESSAGE-RETURNS-LIST ...): 
The previously added rule CHAT-MESSAGE-LIST-P-OF-ADD-MESSAGE subsumes
a newly proposed :REWRITE rule generated from ADD-MESSAGE-RETURNS-LIST,
in the sense that the old rule rewrites a more general target.  Because
the new rule will be tried first, it may nonetheless find application.

Goal'
Subgoal 2
Subgoal 1

Q.E.D.

The storage of ADD-MESSAGE-RETURNS-LIST depends upon the :compound-
recognizer rule TRUE-LISTP-WHEN-CHAT-MESSAGE-LIST-P-COMPOUND-RECOGNIZER
and the :type-prescription rule CHAT-MESSAGE-LIST-P.

Summary
Form:  ( DEFTHM ADD-MESSAGE-RETURNS-LIST ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-CHAT-MESSAGE-P)
        (:COMPOUND-RECOGNIZER
             TRUE-LISTP-WHEN-CHAT-MESSAGE-LIST-P-COMPOUND-RECOGNIZER)
        (:DEFINITION ADD-MESSAGE)
        (:DEFINITION NATP)
        (:EXECUTABLE-COUNTERPART CHAT-MESSAGE-LIST-P)
        (:REWRITE CHAT-MESSAGE-FIX-WHEN-CHAT-MESSAGE-P)
        (:REWRITE CHAT-MESSAGE-LIST

In [36]:
;; Helper: first-is-system-p recognizer for car
(defthm first-is-system-preserved-by-cons
  (implies (let ((role (chat-message->role msg)))
             (chat-role-case role :system t :otherwise nil))
           (first-is-system-p (cons msg rest)))
  :hints (("Goal" :in-theory (enable first-is-system-p))))


ACL2 Warning [Double-rewrite] in ( DEFTHM FIRST-IS-SYSTEM-PRESERVED-BY-CONS
...):  In a :REWRITE rule generated from FIRST-IS-SYSTEM-PRESERVED-BY-CONS,
equivalence relation CHAT-MESSAGE-EQUIV$INLINE is maintained at one
problematic occurrence of variable MSG in the first hypothesis, but
not at any binding occurrence of MSG.  Consider replacing that occurrence
of MSG in the first hypothesis with (DOUBLE-REWRITE MSG).  See :doc
double-rewrite for more information on this issue.

Goal'

Q.E.D.

The storage of FIRST-IS-SYSTEM-PRESERVED-BY-CONS depends upon the :type-
prescription rule FIRST-IS-SYSTEM-P.

Summary
Form:  ( DEFTHM FIRST-IS-SYSTEM-PRESERVED-BY-CONS ...)
Rules: ((:DEFINITION EQL)
        (:DEFINITION FIRST-IS-SYSTEM-P)
        (:EXECUTABLE-COUNTERPART EQUAL)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE CAR-CONS)
        (:TYPE-PRESCRIPTION FIRST-IS-SYSTEM-P))
Warnings:  Double-rewrite
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Prover steps counted: 

In [37]:
;; Theorem: truncate-to-fit preserves system prompt when present
(defthm truncate-preserves-system-prompt
  (implies (and (chat-message-list-p msgs)
                (consp msgs)
                (first-is-system-p msgs))
           (and (consp (truncate-to-fit msgs max-tokens))
                (equal (car (truncate-to-fit msgs max-tokens))
                       (car msgs))))
  :hints (("Goal" :in-theory (enable truncate-to-fit
                                      extract-system-message
                                      first-is-system-p))))


ACL2 Warning [Double-rewrite] in ( DEFTHM TRUNCATE-PRESERVES-SYSTEM-PROMPT
...):  In a :REWRITE rule generated from TRUNCATE-PRESERVES-SYSTEM-PROMPT,
equivalence relations SET-EQUIV and LIST-EQUIV are maintained at one
problematic occurrence of variable MSGS in the second hypothesis, but
not at any binding occurrence of MSGS.  Consider replacing that occurrence
of MSGS in the second hypothesis with (DOUBLE-REWRITE MSGS).  See :doc
double-rewrite for more information on this issue.


ACL2 Warning [Double-rewrite] in ( DEFTHM TRUNCATE-PRESERVES-SYSTEM-PROMPT
...):  In a :REWRITE rule generated from TRUNCATE-PRESERVES-SYSTEM-PROMPT,
equivalence relations SET-EQUIV and LIST-EQUIV are maintained at one
problematic occurrence of variable MSGS in the second hypothesis, but
not at any binding occurrence of MSGS.  Consider replacing that occurrence
of MSGS in the second hypothesis with (DOUBLE-REWRITE MSGS).  See :doc
double-rewrite for more information on this issue.


ACL2 Warning [Double-re